<a href="https://colab.research.google.com/github/felixlys/OptionBarrier-Option/blob/main/MonteCarloSimulationPricingExoticOption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

In [ ]:
#Down and Out Barrier Option with Monte Carlo Simulation
#By : Felix Lyanto Setiawan
Z=np.random.normal(size=(10,4)) #10 adalah jumlah simulasi 4 adalah jmlh timestep dr 0 i=0,1,2,3
r=np.exp(0.08/3)-1
std=0.35
dt=1/3
X=np.zeros([10,4])
for i in range (10):
  for j in range(3):
    X[i,j+1]=(r-0.5*std**2)*(1/3)+std*np.sqrt(dt)*Z[i,j]
S=np.zeros([10,4])
S[:,0]=100
for i in range(10):
  for j in range(3):
    S[i,j+1]=S[i,j]*np.exp(X[i,j+1])
#Barrier elemination
for i in range(10):
  if S[i,-1]<75:
    S[i,-1]=0
#Valuation
ST=S[:,-1]
K=70 #Strike Price
V=np.zeros(10)#Diganti dengan banyaknya simulasi ya bole
for i in range(10):
  V[i]=np.exp(-0.08)* max(ST[i]-K,0)# Klo put dibalik saja K dan ST nya
Price=np.mean(V)
print(Price)

28.980674521475173


In [ ]:
Z= np.random.normal(size=(10,4))
Val=np.zeros([10,4])
Val[:,0]=100
r=np.exp(0.08/3)-1
dt=1/3
std=0.35
for i in range(10):
  for j in range(3):
    Val[i,j+1]=Val[i,j]*np.exp((r-0.5*std**2)*dt+std*np.sqrt(dt)*Z[i,j])
print(Val)
ST=Val[:,-1]
print(ST)
Value=np.exp(-0.08)*np.max(ST-75,0)
print(Value)

[[100.         100.72497375 124.15052299  99.3178679 ]
 [100.          81.15041354  90.76178772 128.59335218]
 [100.         105.39459764 120.32535759 108.51788029]
 [100.          79.67037692  67.91908083  67.47112449]
 [100.          82.69940725 110.49245236 101.18350878]
 [100.          98.17008986 115.27533711  99.85926648]
 [100.          96.21243935 115.94037228 120.69551234]
 [100.          84.12602482  77.43247829  60.32323176]
 [100.         100.30400646 126.93543264 161.94177594]
 [100.          77.35629749  72.75784397  78.1186232 ]]
[ 99.3178679  128.59335218 108.51788029  67.47112449 101.18350878
  99.85926648 120.69551234  60.32323176 161.94177594  78.1186232 ]
80.25737455144626


In [ ]:
#Down and Out Barrier Option with Monte Carlo Simulation
#By : Felix Lyanto Setiawan
def DownandOutMonteCarlo(r,std,Sim,timesteps,S0,B,K):
  Z=np.random.normal(size=(Sim,timesteps))
  X=np.zeros([Sim,timesteps])
  dt=1/timesteps
  for i in range(Sim):
    for j in range(timesteps-1):
      X[i,j+1]=((np.exp(r/timesteps)-1)-0.5*std**2)*(dt)+std*np.sqrt(dt)*Z[i,j]
  S=np.zeros([Sim,timesteps])
  S[:,0]=S0
  for i in range(Sim):
    for j in range(timesteps-1):
      S[i,j+1]=S[i,j]*np.exp(X[i,j+1])
  #Barrier Elemination
  for i in range(Sim):
    if S[i,-1]<B:
      S[i,-1]=0
  #Valuation
  ST=S[:,-1]
  V=np.zeros(Sim)
  for i in range(Sim):
    V[i]=np.exp(-r)*max(ST[i]-K,0)
  Harga=np.mean(V)
  return V,Harga
r = 0.08
std = 0.35
Sim = 1000
timesteps = 365
S0 = 100
B = 75
K = 70
V,Price = DownandOutMonteCarlo(r, std, Sim, timesteps, S0, B, K)

print(Price)

31.323713004313174


In [ ]:
#Vanilla Options
#By : Felix Lyanto Setiawan
import numpy as np
def VanillaMonteCarlo(r,std,Sim,timesteps,S0,K):
  Z=np.random.normal(size=(Sim,timesteps))
  X=np.zeros([Sim,timesteps])
  dt=1/timesteps
  for i in range(Sim):
    for j in range(timesteps-1):
      X[i,j+1]=((np.exp(r/timesteps)-1)-0.5*std**2)*(dt)+std*np.sqrt(dt)*Z[i,j]
  S=np.zeros([Sim,timesteps])
  S[:,0]=S0
  for i in range(Sim):
    for j in range(timesteps-1):
      S[i,j+1]=S[i,j]*np.exp(X[i,j+1])
  #Valuation
  ST=S[:,-1]
  V=np.zeros(Sim)
  for i in range(Sim):
    V[i]=np.exp(-r)*max(ST[i]-K,0)
  Harga=np.mean(V)
  return V,Harga
r = 0.08
std = 0.35
Sim = 1000
timesteps = 365
S0 = 100
B = 75
K = 70
V,Price = VanillaMonteCarlo(r, std, Sim, timesteps, S0, K)

print(Price)

29.146129591163888


In [ ]:
#Blackscholes Formula
#Felix Lyanto Setiawan
def BlackscholesFormula(Data,K):

